In [1]:
import requests
from html.parser import HTMLParser
import json
import pandas as pd
from bs4 import BeautifulSoup
import re 
import pandas as pd

def return_response(url):
    headers = {
            'sec-fetch-mode': 'cors',
            'accept-language': 'en-US,en;q=0.9',
            'authorization': '',
            'x-requested-with': 'XMLHttpRequest',
            'appos': 'web',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
            'accept': '*/*',
            'authority': 'stockx.com',
            'sec-fetch-site': 'same-origin',
            'appversion': '0.1',
    'accept': 'application/json'}
    
    return requests.get(url, headers=headers)

def return_base_json(page_response):
    soup = BeautifulSoup(page_response.content, "html.parser")
    data = soup.findAll('script', type='application/ld+json')
    data_skus = data[4]
    extracted_data = data_skus.contents
    extracted_data = extracted_data[0] 
    json_data = json.loads(extracted_data)
    
    return json_data

def return_base_data(page_response):
    base_data = []
    json_data = return_base_json(page_response)
    shoe_data = json_data['itemListElement']
    
    for i, shoe in enumerate(shoe_data):
        data = shoe['item']
        base_data.append({key: data[key] for key in data.keys()
                          & {'name', 'releaseDate', 'brand', 'model', 'color', 'itemCondition', 'sku', 'image'}})
        if 'offers' in data:
            base_data[i]['url'] = data['offers']['url']
        else:  # some shoe urls are missing, going to have to do these manually
            base_data[i]['url'] = 'missing'
    
    return base_data

In [2]:
base_data = []

for page in range(1,26):
    url = f'https://stockx.com/nike/?page={page}'
    resp = return_response(url)
    page_base_data = return_base_data(resp)
    base_data.append(page_base_data)

flat_base_data = [item for sublist in base_data for item in sublist]
df = pd.DataFrame(flat_base_data)
df.head()

,brand,color,image,itemCondition,model,name,releaseDate,sku,url
0,Nike,Infrared/Black-Dark Sage-Baroque Brown,https://stockx.imgix.net/Nike-Air-Max-90-Rever...,https://schema.org/NewCondition,Nike Air Max 90,Nike Air Max 90 Reverse Duck Camo (2020),2020-03-26 23:59:59,ee92167b-b780-45d9-800a-c52749ee0ccb,https://stockx.com/nike-air-max-90-reverse-duc...
1,Nike,White/White,https://stockx.imgix.net/Nike-LD-Waffle-Sacai-...,https://schema.org/NewCondition,Nike LD Waffle,Nike LD Waffle Sacai White Nylon,2020-03-10 23:59:59,42957d33-ff87-4377-972a-d755ee47f7f9,https://stockx.com/nike-ld-waffle-sacai-white-...
2,Nike,White/White-White,https://stockx.imgix.net/Nike-Air-Force-1-Low-...,https://schema.org/NewCondition,Nike Air Force 1 Low,Nike Air Force 1 Low Supreme White,2020-03-05 23:59:59,5b85a42d-daed-44a8-b75e-42ec85fe41af,https://stockx.com/nike-air-force-1-low-suprem...
3,Nike,Black/Orange-Cement Grey-Outdoor green,https://stockx.imgix.net/Nike-SB-Dunk-Low-Safa...,https://schema.org/NewCondition,Nike SB Dunk Low,Nike SB Dunk Low Safari,2020-03-14 23:59:59,ba80c366-c551-449e-9da4-75d7ce26c49b,https://stockx.com/nike-sb-dunk-low-atmos-safari
4,Nike,Black/Black,https://stockx.imgix.net/Nike-LD-Waffle-Sacai-...,https://schema.org/NewCondition,Nike LD Waffle,Nike LD Waffle Sacai Black Nylon,2020-03-10 23:59:59,223a7009-fc66-4056-be6d-470add8bcaeb,https://stockx.com/nike-ld-waffle-sacai-black-...
